In [2]:
import numpy as np
import operator, sys, os
import tensorflow as tf
import plotting
import itertools
import matplotlib
import collections
import random
import pandas as pd
import operator
matplotlib.style.use('ggplot')
import gym
import numpy as np 
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import Add, Multiply
from keras.optimizers import Adam
import keras.backend as K

import tensorflow as tf

import random
from collections import deque

Using TensorFlow backend.


In [7]:
action_map = pd.read_csv("latest_actions_30.07.2018.csv")
action_map

,Unnamed: 0,action,state,qid,item_class,item_class_id
0,0,open_life1,open,0,life,0.0
1,1,open_life2,open,1,life,0.0
2,2,open_life3,open,2,life,0.0
3,3,open_life4,open,3,life,0.0
4,4,open_life5,open,4,life,0.0
5,5,open_car1,open,5,car,1.0
6,6,open_car2,open,6,car,1.0
7,7,open_car3,open,7,car,1.0
8,8,open_car4,open,8,car,1.0
9,9,open_car5,open,9,car,1.0


In [ ]:
class User:
    def __init__(self, user_class, item_class, action_map):
        self.name="Simulated-user"
        self.user_class = user_class
        self.item_class = item_class
        self.action_map = action_map
        print("---------------New User--------------------")
        if self.user_class == 0:
            uc_string = 'Young Adult'
        if self.user_class == 1:
            uc_string = "Middle Age"
        if self.user_class == 2:
            uc_string = "Old Age"
        if self.item_class == 0:
            ic_string = "Life"
        if self.item_class == 1:
            ic_string = "Car"            
        if self.item_class == 2:
            ic_string = "Dental"            
        if self.item_class == 3:
            ic_string = "Health"            
        if self.item_class == 4:
            ic_string = "Disability"
        print("ITEM CLASS :",ic_string)
        self.state = 0
        self.prev_response = 0
        self.action_map = pd.read_csv("latest_actions_30.07.2018.csv")
        self.done = 0
        self.current_reward = 0
        self.allowed_actions = np.array([])
        
    def print_user_speech(self):
        if self.prev_response == 1:
            print("USER SAID YES")
        elif self.prev_response == 0:
            print("USER SAID NO")
            
    def get_allowed_actions(self):
  
        if (self.state == 0 ):
            self.allowed_actions = np.array(self.action_map[(self.action_map['item_class_id']== self.item_class)& (self.action_map['state']== 'open')]['qid'].values)
            return
        elif (self.state in [1,2,3,4,5]):
            self.allowed_actions = np.array(self.action_map[(self.action_map['item_class_id']== self.item_class)& (self.action_map['state']== 'ask')]['qid'].values)
            return
        elif (self.state in [6,7,8,9,12]):
            self.allowed_actions = np.array(self.action_map[(self.action_map['state']== 'close')]['qid'].values)
            return
        else:
            print("Dialog state is wrong or terminal state is reached. Current State while getting actions",self.state)
            self.allowed_actions = np.array([]) 
            return
    
    def get_reward(self):
        if (self.state == 10):
            self.current_reward = 1
#         elif(self.state == 11):
#             self.current_reward = -0.01
        else:
            self.current_reward = 0
        return
    def user_reset(self):
        self.state = 0
        self.prev_response = 0
        self.action_map = pd.read_csv("latest_actions_30.07.2018.csv")
        self.done = 0
        self.current_reward = 0
        self.allowed_actions = np.array([])
        
    def is_done(self):
        if (self.state == 10):
            self.done = 1
            return
        elif (self.state == 11):
            self.done = 1
            return
        
    def respond(self,action):
        print("Bot Says : ",action_map[action_map['qid']==action]['action'].values)
        
        if (self.state == 0):
            self.get_allowed_actions()
            if (self.allowed_actions[0]==action):
                self.prev_response = np.random.choice([1,0], p=[0.9,0.1])
                if self.prev_response:
                    self.state = 1
                else:
                    self.state = 11
            elif (self.allowed_actions[1]==action):
                self.prev_response = np.random.choice([1,0], p=[0.2,0.8])  
                if self.prev_response:
                    self.state = 2
                else:
                    self.state = 11
            elif (self.allowed_actions[2]==action):
                self.prev_response = np.random.choice([1,0], p=[0.2,0.8])
                if self.prev_response:
                    self.state = 3
                else:
                    self.state = 11
            elif (self.allowed_actions[3]==action):
                self.prev_response = np.random.choice([1,0], p=[0.3,0.7])
                if self.prev_response:
                    self.state = 4
                else:
                    self.state = 11
            elif (self.allowed_actions[4]==action):
                self.prev_response = np.random.choice([1,0], p=[0.2,0.8])
                if self.prev_response:
                    self.state = 5
                else:
                    self.state = 11
            else:
                print("Something wrong with chosen action",action)
            
            self.print_user_speech()
            self.get_reward()
            self.is_done()
            return
        
        if (self.state in [1,2,3,4,5]):
            self.get_allowed_actions()
            if (self.allowed_actions[0]==action):
                self.prev_response = np.random.choice([1,0], p=[0.3,0.7])
                if self.prev_response:
                    self.state = 6
                else:
                    self.state = 11
            elif (self.allowed_actions[1]==action):
                self.prev_response = np.random.choice([1,0], p=[0.9,0.1])  
                if self.prev_response:
                    self.state = 7
                else:
                    self.state = 11
            elif (self.allowed_actions[2]==action):
                self.prev_response = np.random.choice([1,0], p=[0.1,0.9])
                if self.prev_response:
                    self.state = 8
                else:
                    self.state = 11
            elif (self.allowed_actions[3]==action):
                self.prev_response = np.random.choice([1,0], p=[0.1,0.9])
                if self.prev_response:
                    self.state = 9
                else:
                    self.state = 11
            elif (self.allowed_actions[4]==action):
                self.prev_response = np.random.choice([1,0], p=[0.1,0.9])
                if self.prev_response:
                    self.state = 12
                else:
                    self.state = 11
            else:
                print("Something wrong with chosen action")
            
            self.print_user_speech()
            self.get_reward()
            self.is_done()
            return
  
        
        if (self.state in [6,7,8,9,12]):
            self.get_allowed_actions()
            if (self.allowed_actions[0]==action):
                self.prev_response = np.random.choice([1,0], p=[0.3,0.7])
                if self.prev_response:
                    self.state = 10
                else:
                    self.state = 11
            elif (self.allowed_actions[1]==action):
                self.prev_response = np.random.choice([1,0], p=[0.2,0.8])  
                if self.prev_response:
                    self.state = 10
                else:
                    self.state = 11
            elif (self.allowed_actions[2]==action):
                self.prev_response = np.random.choice([1,0], p=[0.1,0.9])
                if self.prev_response:
                    self.state = 10
                else:
                    self.state = 11
            elif (self.allowed_actions[3]==action):
                self.prev_response = np.random.choice([1,0], p=[0.2,0.8])
                if self.prev_response:
                    self.state = 10
                else:
                    self.state = 11
            elif (self.allowed_actions[4]==action):
                self.prev_response = np.random.choice([1,0], p=[0.9,0.1])
                if self.prev_response:
                    self.state = 10
                else:
                    self.state = 11
            else:
                print("Something wrong with chosen action")
            
            self.print_user_speech()
            self.get_reward()
            self.is_done()
            return

In [ ]:
class Environment:
    def __init__(self):
        self.name="Chat-Environment"
        self.user = User(0,np.random.randint(0,5, size=1)[0],action_map)
        self.N_actions = len(self.user.action_map['qid'].values)
        print("No. actions:", self.N_actions)
        self.state_dim = len([self.user.item_class,self.user.state])
            
    def allowed_actions_with_prob(self, action_probs):
        allowed_action_prob_dict = {}
        self.user.get_allowed_actions()
        for action in self.user.allowed_actions:
            allowed_action_prob_dict[action] = action_probs[action]
        print("Allowed Actions",self.user.allowed_actions)
        print("Allowed Actions Prob Dict",allowed_action_prob_dict)
        return self.user.allowed_actions, allowed_action_prob_dict
        

    def reset(self):
        self.user = User(0,np.random.randint(0,5, size=1)[0],action_map)
        return np.array([self.user.item_class,self.user.state])
        
    def step(self, action):
        print("Bot said ",action)
        self.user.respond(action)
        return np.array([self.user.item_class,self.user.state]), self.user.done, self.user.current_reward


In [ ]:
# class PolicyEstimator():
#     """
#     Policy Function approximator. 
#     """
    
#     def __init__(self, learning_rate=0.01, scope="policy_estimator"):
#         with tf.variable_scope(scope):
#             self.state = tf.placeholder(tf.float32, [None,2], "state")
#             self.action = tf.placeholder(dtype=tf.int32, name="action")
#             self.target = tf.placeholder(dtype=tf.float32, name="target")

# #             self.state_one_hot = tf.one_hot(self.state, 13)
# #             print("ONE HOT STATE", self.state_one_hot)


#             self.h1 = tf.layers.dense(self.state, 8, activation=tf.nn.relu, kernel_initializer=tf.zeros_initializer())
#             self.h2 = tf.layers.dense(self.h1, 16, activation=tf.nn.relu, kernel_initializer=tf.zeros_initializer())
#             self.h3 = tf.layers.dense(self.h2, 8, activation=tf.nn.relu, kernel_initializer=tf.zeros_initializer())
#             self.output_layer = tf.layers.dense(self.h3, env.N_actions, kernel_initializer=tf.zeros_initializer())
#             print("OUTPUT LAYER",self.output_layer)

#             self.action_probs = tf.squeeze(tf.nn.softmax(self.output_layer))
#             self.picked_action_prob = tf.gather(self.action_probs, self.action)

#             # Loss and train op
#             self.loss = -tf.log(self.picked_action_prob) * self.target

#             self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
#             self.train_op = self.optimizer.minimize(
#                 self.loss, global_step=tf.train.get_global_step())
    
#     def predict(self, state, sess=None):
#         sess = sess or tf.get_default_session()
#         return sess.run(self.action_probs, { self.state: state.reshape(1,2) })

#     def update(self, state, target, action, sess=None):
#         sess = sess or tf.get_default_session()
#         feed_dict = { self.state: state.reshape(1,2), self.target: target, self.action: action}
#         _, loss = sess.run([self.train_op, self.loss], feed_dict)
#         return loss

In [ ]:
# class ValueEstimator():
#     """
#     Value Function approximator. 
#     """
    
#     def __init__(self, learning_rate=0.01, scope="value_estimator"):
#         with tf.variable_scope(scope):
#             self.state = tf.placeholder(tf.float32, [None,2], "state")
#             self.target = tf.placeholder(dtype=tf.float32, name="target")


            
# #             self.state_one_hot = tf.one_hot(self.state, 13)

#             self.h1 = tf.layers.dense(self.state, 8, activation=tf.nn.relu, kernel_initializer=tf.zeros_initializer())
#             self.h2 = tf.layers.dense(self.h1, 16, activation=tf.nn.relu, kernel_initializer=tf.zeros_initializer())
#             self.h3 = tf.layers.dense(self.h2, 8, activation=tf.nn.relu, kernel_initializer=tf.zeros_initializer())
#             self.output_layer = tf.layers.dense(self.h3, 1, kernel_initializer=tf.zeros_initializer())



#             self.value_estimate = tf.squeeze(self.output_layer)
#             self.loss = tf.squared_difference(self.value_estimate, self.target)

#             self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
#             self.train_op = self.optimizer.minimize(
#                 self.loss, global_step=tf.train.get_global_step())        
    
#     def predict(self, state, sess=None):
#         sess = sess or tf.get_default_session()
#         return sess.run(self.value_estimate, { self.state: state.reshape(1,2) })

#     def update(self, state, target, sess=None):
#         sess = sess or tf.get_default_session()
#         feed_dict = { self.state: state.reshape(1,2), self.target: target }
#         _, loss = sess.run([self.train_op, self.loss], feed_dict)
#         return loss

In [ ]:
# def actor_critic(env, estimator_policy, estimator_value, num_episodes, discount_factor=1.0):
#     """
#     Actor Critic Algorithm. Optimizes the policy 
#     function approximator using policy gradient.
    
#     Args:
#         env: OpenAI environment.
#         estimator_policy: Policy Function to be optimized 
#         estimator_value: Value function approximator, used as a critic
#         num_episodes: Number of episodes to run for
#         discount_factor: Time-discount factor
    
#     Returns:
#         An EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
#     """

#     # Keeps track of useful statistics
#     stats = plotting.EpisodeStats(
#         episode_lengths=np.zeros(num_episodes),
#         episode_rewards=np.zeros(num_episodes),
#         episode_policy_loss=np.zeros(num_episodes),
#         episode_value_loss=np.zeros(num_episodes))    
    
#     Transition = collections.namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])
    
#     for i_episode in range(num_episodes):
#         # Reset the environment and pick the fisrst action
#         print("\n---------------Next Episode",i_episode," --------------------")
#         state = env.reset()
        

        
#         episode = []
        
#         # One step in the environment
#         for t in itertools.count():
            
            
#             action_probs = estimator_policy.predict(state)
#             print(action_probs.shape)
#             print("Action Probs from Policy",action_probs)
#             allowed_actions, allowed_action_prob_dict = env.allowed_actions_with_prob(action_probs)   
#             expl_chance = np.random.choice([1,0], p=[0.1,0.9])
#             if (expl_chance == 1):
#                 action = np.random.choice(allowed_actions)
#             else:
#                 action = max(allowed_action_prob_dict.items(), key=operator.itemgetter(1))[0]
                

#             next_state, done, reward = env.step(action)
            
#             print("\nNew State",next_state)
#             print("\nStep Rewards", reward)
#             print("DONE?",done)

            
#             # Keep track of the transition
#             episode.append(Transition(
#               state=state, action=action, reward=reward, next_state=next_state, done=done))
            
#             # Update statistics
#             stats.episode_rewards[i_episode] += reward
#             stats.episode_lengths[i_episode] = t
            
#             # Calculate TD Target
#             value_next = estimator_value.predict(next_state)
#             td_target = reward + discount_factor * value_next
#             td_error = td_target - estimator_value.predict(state)
            
#             # Update the value estimator
#             v_loss = estimator_value.update(state, td_target)
            
#             # Update the policy estimator
#             # using the td error as our advantage estimate
#             p_loss = estimator_policy.update(state, td_error, action)
#             stats.episode_policy_loss[i_episode] += p_loss
#             stats.episode_value_loss[i_episode] += v_loss
#             # Print out which step we're on, useful for debugging.
#             print("\rStep {} @ Episode {}/{} ({})".format(
#                     t, i_episode , num_episodes, stats.episode_rewards[i_episode]), end="")

#             if done:
#                 print("\nFinal Dialog State", env.user.state)
#                 print("---------------Episode DONE--------------------")
#                 break
                
#             state = next_state
    
#     return stats

In [ ]:
tf.reset_default_graph()
env = Environment()

policy_estimator = PolicyEstimator()
value_estimator = ValueEstimator()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    stats = actor_critic(env, policy_estimator, value_estimator, 5000, 0.90)

In [ ]:
plotting.plot_episode_stats(stats, smoothing_window=100)

In [ ]:
"""
solving pendulum using actor-critic model
"""



# determines how to assign values to each state, i.e. takes the state
# and action (two-input model) and determines the corresponding value
class ActorCritic:
    def __init__(self, env, sess):
        self.env  = env
        self.sess = sess

        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = .995
        self.gamma = .95
        self.tau   = .125

        # ===================================================================== #
        #                               Actor Model                             #
        # Chain rule: find the gradient of chaging the actor network params in  #
        # getting closest to the final value network predictions, i.e. de/dA    #
        # Calculate de/dA as = de/dC * dC/dA, where e is error, C critic, A act #
        # ===================================================================== #

        self.memory = deque(maxlen=2000)
        self.actor_state_input, self.actor_model = self.create_actor_model()
        _, self.target_actor_model = self.create_actor_model()

        self.actor_critic_grad = tf.placeholder(tf.float32, 
            [None, self.env.N_actions]) # where we will feed de/dC (from critic)
        
        actor_model_weights = self.actor_model.trainable_weights
        self.actor_grads = tf.gradients(self.actor_model.output, 
            actor_model_weights, -self.actor_critic_grad) # dC/dA (from actor)
        grads = zip(self.actor_grads, actor_model_weights)
        self.optimize = tf.train.AdamOptimizer(self.learning_rate).apply_gradients(grads)

        # ===================================================================== #
        #                              Critic Model                             #
        # ===================================================================== #       

        self.critic_state_input, self.critic_action_input, \
            self.critic_model = self.create_critic_model()
        _, _, self.target_critic_model = self.create_critic_model()

        self.critic_grads = tf.gradients(self.critic_model.output, 
            self.critic_action_input) # where we calcaulte de/dC for feeding above
        
        # Initialize for later gradient calculations
        self.sess.run(tf.initialize_all_variables())

    # ========================================================================= #
    #                              Model Definitions                            #
    # ========================================================================= #

    def create_actor_model(self):
        state_input = Input(shape=self.env.state_dim)
        h1 = Dense(24, activation='relu')(state_input)
        h2 = Dense(48, activation='relu')(h1)
        h3 = Dense(24, activation='relu')(h2)
        output = Dense(self.env.N_actions, activation='relu')(h3)
        
        model = Model(input=state_input, output=output)
        adam  = Adam(lr=0.001)
        model.compile(loss="mse", optimizer=adam)
        return state_input, model

    def create_critic_model(self):
        state_input = Input(shape=self.env.state_dim)
        state_h1 = Dense(24, activation='relu')(state_input)
        state_h2 = Dense(48)(state_h1)
        
        action_input = Input(shape=self.env.N_actions)
        action_h1    = Dense(48)(action_input)
        
        merged    = Add()([state_h2, action_h1])
        merged_h1 = Dense(24, activation='relu')(merged)
        output = Dense(1, activation='relu')(merged_h1)
        model  = Model(input=[state_input,action_input], output=output)
        
        adam  = Adam(lr=0.001)
        model.compile(loss="mse", optimizer=adam)
        return state_input, action_input, model

    # ========================================================================= #
    #                               Model Training                              #
    # ========================================================================= #

    def remember(self, cur_state, action, reward, new_state, done):
        self.memory.append([cur_state, action, reward, new_state, done])

    def _train_actor(self, samples):
        for sample in samples:
            cur_state, action, reward, new_state, _ = sample
            predicted_action = self.actor_model.predict(cur_state)
            grads = self.sess.run(self.critic_grads, feed_dict={
                self.critic_state_input:  cur_state,
                self.critic_action_input: predicted_action
            })[0]

            self.sess.run(self.optimize, feed_dict={
                self.actor_state_input: cur_state,
                self.actor_critic_grad: grads
            })
            
    def _train_critic(self, samples):
        for sample in samples:
            cur_state, action, reward, new_state, done = sample
            if not done:
                target_action = self.target_actor_model.predict(new_state)
                future_reward = self.target_critic_model.predict(
                    [new_state, target_action])[0][0]
                reward += self.gamma * future_reward
            self.critic_model.fit([cur_state, action], reward, verbose=0)
        
    def train(self):
        batch_size = 32
        if len(self.memory) < batch_size:
            return

        rewards = []
        samples = random.sample(self.memory, batch_size)
        self._train_critic(samples)
        self._train_actor(samples)

    # ========================================================================= #
    #                         Target Model Updating                             #
    # ========================================================================= #

    def _update_actor_target(self):
        actor_model_weights  = self.actor_model.get_weights()
        actor_target_weights = self.target_critic_model.get_weights()
        
        for i in range(len(actor_target_weights)):
            actor_target_weights[i] = actor_model_weights[i]
        self.target_critic_model.set_weights(actor_target_weights)

    def _update_critic_target(self):
        critic_model_weights  = self.critic_model.get_weights()
        critic_target_weights = self.critic_target_model.get_weights()
        
        for i in range(len(critic_target_weights)):
            critic_target_weights[i] = critic_model_weights[i]
        self.critic_target_model.set_weights(critic_target_weights)     

    def update_target(self):
        self._update_actor_target()
        self._update_critic_target()

    # ========================================================================= #
    #                              Model Predictions                            #
    # ========================================================================= #

    def act(self, cur_state):
        self.epsilon *= self.epsilon_decay
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        return self.actor_model.predict(cur_state)


In [ ]:

def main():
	sess = tf.Session()
	K.set_session(sess)
	env = gym.make("Pendulum-v0")
	actor_critic = ActorCritic(env, sess)

	num_trials = 10000
	trial_len  = 500

	cur_state = env.reset()
	action = env.action_space.sample()
	while True:
		env.render()
		cur_state = cur_state.reshape((1, env.observation_space.shape[0]))
		action = actor_critic.act(cur_state)
		action = action.reshape((1, env.action_space.shape[0]))

		new_state, reward, done, _ = env.step(action)
		new_state = new_state.reshape((1, env.observation_space.shape[0]))

		actor_critic.remember(cur_state, action, reward, new_state, done)
		actor_critic.train()

		cur_state = new_state

if __name__ == "__main__":
	main()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/Users/shishir/anaconda3/envs/BotconnectSalesAI/lib/python3.6/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
/Users/shishir/anaconda3/envs/BotconnectSalesAI/lib/python3.6/site-packages/ipykernel_launcher.py:80: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [10]:
action_map.drop(['Unnamed: 0'], axis=1)

,action,state,qid,item_class,item_class_id
0,open_life1,open,0,life,0.0
1,open_life2,open,1,life,0.0
2,open_life3,open,2,life,0.0
3,open_life4,open,3,life,0.0
4,open_life5,open,4,life,0.0
5,open_car1,open,5,car,1.0
6,open_car2,open,6,car,1.0
7,open_car3,open,7,car,1.0
8,open_car4,open,8,car,1.0
9,open_car5,open,9,car,1.0


In [31]:
a = np.array(action_map[(action_map['item_class_id']== 0) | (action_map['item_class']== 'all')]['action'].values)

In [32]:
len(a)

15

In [20]:
env = gym.make("Pendulum-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [23]:
env.action_space.shape

(1,)

In [29]:
env.observation_space.shape

(3,)